In [2]:
%matplotlib inline
import lightroot

In [ ]:
sample_time_frame = 0
stack = lightroot.io.get_stack(sample_time_frame)
#max_int = lightroot.io.get_max_int(sample_time_frame)

In [ ]:
#blobs=lightroot.blobs.detect(stack) does the following steps
sharpened = lightroot.blobs.sharpen(stack)
lightroot.io.plotimg(sharpened)

In [ ]:
markers = lightroot.blobs.blob_labels(sharpened)
blobs = lightroot.blobs.blob_centroids(markers)
lightroot.io.overlay_blobs(stack,blobs)

In [ ]:
n = 450
all_blobs = lightroot.process_files(n)
lightroot.tracks_from_blobs(all_blobs, n,"./cached_data/{}.png")